### Imports & Datatset Prep

In [2]:
# importing libraries
from os import listdir
from collections import Counter

import skimage
import numpy as np
from tifffile import imsave, imread
import glob
from timeit import default_timer as timer
import os.path

import subprocess
import json
import io
from PIL import Image
from math import sqrt, exp, log
from matplotlib import cm
from matplotlib import pyplot as plt

import csv

In [2]:
%run "stitching pipeline.ipynb"

In [3]:
%run "evaluation pipeline.ipynb"

In [ ]:
dataset_1 = "../datasets/1"
dataset_2 = "../datasets/2"
dataset_3 = "../datasets/3"
dataset_4 = "../datasets/4"
dataset_5 = "../datasets/5"
dataset_6 = "../datasets/6"
dataset_7 = "../datasets/7"
dataset_8 = "../datasets/8"
dataset_9 = "../datasets/9"
dataset_10 = "../datasets/10"
dataset_11 = "../datasets/11"
dataset_12 = "../datasets/12"
dataset_13 = "../datasets/13"
dataset_14 = "../datasets/14"
dataset_15 = "../datasets/15"
dataset_16 = "../datasets/16"

dirs = [dataset_1, dataset_2, dataset_4, dataset_5 , dataset_6, dataset_7, dataset_8, dataset_9, 
        dataset_10, dataset_11, dataset_12, dataset_13, dataset_14]

### Helpers

In [5]:
%%time
def raw_to_thermal(filename):
    exiftool_path = '../exiftool.exe'

    """
    extracts the thermal image as 2D numpy array with temperatures in oC
    """

    # read image metadata needed for conversion of the raw sensor values
    # E=1,SD=1,RTemp=20,ATemp=RTemp,IRWTemp=RTemp,IRT=1,RH=50,PR1=21106.77,PB=1501,PF=1,PO=-7340,PR2=0.012545258
    meta_json = subprocess.check_output(
        [exiftool_path, filename, '-Emissivity', '-SubjectDistance', '-AtmosphericTemperature',
         '-ReflectedApparentTemperature', '-IRWindowTemperature', '-IRWindowTransmission', '-RelativeHumidity',
         '-PlanckR1', '-PlanckB', '-PlanckF', '-PlanckO', '-PlanckR2', '-j','-Model'])
    meta = json.loads(meta_json.decode())[0]

    # exifread can't extract the embedded thermal image, use exiftool instead
    thermal_img_bytes = subprocess.check_output([exiftool_path, "-RawThermalImage", "-b", filename])
    thermal_img_stream = io.BytesIO(thermal_img_bytes)

    thermal_img = Image.open(thermal_img_stream)
    thermal_np = np.array(thermal_img)
    return thermal_np

CPU times: total: 0 ns
Wall time: 0 ns


In [6]:
def extract_paths(dir):
    dataset = []
    thermals = []
    for image in os.listdir(dir):
        if (image.endswith(".JPG") or image.endswith(".jpg")):
            dataset += [f'{dir}/{image}']
    for i in range(len(dataset)):
        thermals += [raw_to_thermal(dataset[i])]
    return (dataset, thermals)

def save_to_csv(data, filename):
    for i in range(len(data)-2):
            if (i>2): data[i] = [f"datatset {i+2}"] + data[i]
            else: data[i] = [f"datatset {i+1}"] + data[i]
    data[-2] = [f"aggregates"] + data[-2]
    data[-1] = [f"fails"] + data[-1]
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)

In [7]:
# each of the following functions calculates the color map 
# range for a sequence of thermal inages as per a particular strategy

# averaging min and max values
def get_cmap_avg(thermals):
    mins = []
    maxs = []
    for image in thermals:
    
        cmap_min = image.min()
        cmap_max = image.max()
        mins += [cmap_min]
        maxs += [cmap_max]
        
    return (np.mean(mins), np.mean(maxs))

# selecting absolute min and max
def get_cmap_full(thermals):
    mins = []
    maxs = []
    for image in thermals:
        
        cmap_min = image.min()
        cmap_max = image.max()
        mins += [cmap_min]
        maxs += [cmap_max]
        
    return (min(mins), max(maxs))

# restricting the range based on frequency of values
def get_cmap_auto(thermals):
    mins = []
    maxs = []
    for image in thermals:            
        # Flatten the image array
        pixel_values = image.copy().flatten()
        pixel_counts = Counter(pixel_values)
        
        # Get the most common pixel values
        pixel_counts = pixel_counts.most_common()
        most_frequent_values = [pixel[0] for pixel in pixel_counts[:70]]
        
        # Find the range (min and max) of these values
        cmap_min = min(most_frequent_values)
        cmap_max = max(most_frequent_values)
        mins += [cmap_min]
        maxs += [cmap_max]
        
        cmap_range = (cmap_min, cmap_max)
    return (min(mins), max(maxs))

In [8]:
# extracts and saves thermal images
def color_map(dir, thermals, range_fun, cmap, path_suf = ""):
    thermal_paths = []
    try:
       os.makedirs(f'{dir}/cmap{cmap}{path_suf}')
    except FileExistsError:
       pass
    (vmin, vmax) = range_fun(thermals)
    for i in range(len(thermals)):
        plt.imsave(f'{dir}/cmap{cmap}{path_suf}/{cmap}_{i}.JPG', thermals[i], cmap=cmap, vmin=vmin, vmax=vmax)
        thermal_paths += [f'{dir}/cmap{cmap}{path_suf}/{cmap}_{i}.JPG']
    return thermal_paths

In [9]:
def aggregate_results(eval_results, filename):
    results = []
    fails = 0
    for i in eval_results:
        entropy_vals, clarity_vals, psnr_vals, ssim_vals, mi_vals = [], [], [], [], []
        fails += np.sum(np.array([0 if x is not None else 1 for x in i]))
        i = np.array([x if x is not None else [0, 0, 0, 0, 0] for x in i])
        for j in i:
            entropy_vals.append(j[0])
            clarity_vals.append(j[1])
            psnr_vals.append(j[2])
            ssim_vals.append(j[3])
            mi_vals.append(j[4])
        results.append([np.mean(entropy_vals), np.mean(clarity_vals), np.mean(psnr_vals), np.mean(ssim_vals), np.mean(mi_vals)])
    
    entropy_f, clarity_f, psnr_f, ssim_f, mi_f = [], [], [], [], []
    for j in np.nan_to_num(results, nan=0.0):
        entropy_f.append(j[0])
        clarity_f.append(j[1])
        psnr_f.append(j[2])
        ssim_f.append(j[3])
        mi_f.append(j[4])
    results.append([np.mean(entropy_f), np.mean(clarity_f), np.mean(psnr_f), np.mean(ssim_f), np.mean(mi_f)])
    results.append([fails])

    save_to_csv(results, filename)
    return results

### Main Routine

In [10]:
# given a function to restrict the cmap range, a cmap, and a directory
# extracts and creates the thermal images as per the cmap specifications 
# then runs the stitching and evaluation pipelines on said directory 
def main_routine(range_fun, cmap, result_dir):
    results = []
    int = 0
    for dir in dirs:
        dir_metrics = []
        dataset, thermals = extract_paths(dir)
        thermals = color_map(dir, thermals, range_fun, cmap)
        for i in range(1, len(dataset)):
            (ref_matches, reference) = ground_truth(dataset[i], dataset[i-1], thermals[i], thermals[i-1], int, result_dir)
            (img_matches, mosaic) = stitch(thermals[i], thermals[i-1], (int+1), result_dir)
    
            if (ref_matches == None or img_matches == None or reference == None or mosaic == None): 
                dir_metrics += [None]
                pass
            else:
                metrics = evaluate_mosaic(reference, mosaic, ref_matches, img_matches)
                dir_metrics += [metrics]
            int += 2
        results += [dir_metrics]
    return results

### Running Routine for all Cmaps and Dirs

In [ ]:
eval_jet_full = main_routine(get_cmap_full, "jet", f"../datasets/stitches_jet_full")
eval_jet_avg = main_routine(get_cmap_avg, "jet", f"../datasets/stitches_jet_avg")
eval_jet_auto = main_routine(get_cmap_auto, "jet", f"../datasets/stitches_jet_auto")
results_jet_full = aggregate_results(eval_results_full, "eval_results/jet_full.csv")
results_jet_avg = aggregate_results(eval_results_avg, "eval_results/jet_avg.csv")
results_jet_auto = aggregate_results(eval_results_auto, "eval_results/jet_auto.csv")

No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
No keypoints detected.
No keypoints detected.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints dete

In [ ]:
eval_inferno_full = main_routine(get_cmap_full, "inferno", f"../datasets/stitches_inferno_full")
eval_inferno_avg = main_routine(get_cmap_avg, "inferno", f"../datasets/stitches_inferno_avg")
eval_inferno_auto = main_routine(get_cmap_auto, "inferno", f"../datasets/stitches_inferno_auto")
results_inferno_full = aggregate_results(eval_inferno_full, "eval_results/inferno_full.csv")
results_inferno_avg = aggregate_results(eval_inferno_avg, "eval_results/inferno_avg.csv")
results_inferno_auto = aggregate_results(eval_inferno_auto, "eval_results/inferno_auto.csv")

No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
No keypoints detected.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches were found.
Not enough

In [ ]:
eval_plasma_full = main_routine(get_cmap_full, "plasma", f"../datasets/stitches_plasma_full")
eval_plasma_avg = main_routine(get_cmap_avg, "plasma", f"../datasets/stitches_plasma_avg")
eval_plasma_auto = main_routine(get_cmap_auto, "plasma", f"../datasets/stitches_plasma_auto")
results_plasma_full = aggregate_results(eval_plasma_full, "eval_results/plasma_full.csv")
results_plasma_avg = aggregate_results(eval_plasma_avg, "eval_results/plasma_avg.csv")
results_plasma_auto = aggregate_results(eval_plasma_auto, "eval_results/plasma_auto.csv")

No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No 

In [ ]:
eval_hot_full = main_routine(get_cmap_full, "hot", f"../datasets/stitches_hot_full")
eval_hot_avg = main_routine(get_cmap_avg, "hot", f"../datasets/stitches_hot_avg")
eval_hot_auto = main_routine(get_cmap_auto, "hot", f"../datasets/stitches_hot_auto")
results_hot_full = aggregate_results(eval_hot_full, "eval_results/hot_full.csv")
results_hot_avg = aggregate_results(eval_hot_avg, "eval_results/hot_avg.csv")
results_hot_auto = aggregate_results(eval_hot_auto, "eval_results/hot_auto.csv")

No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints det

In [ ]:
eval_magma_full = main_routine(get_cmap_full, "inferno", f"../datasets/stitches_magma_full")
eval_magma_avg = main_routine(get_cmap_avg, "inferno", f"../datasets/stitches_magma_avg")
eval_magma_auto = main_routine(get_cmap_auto, "inferno", f"../datasets/stitches_magma_auto")
results_magma_auto = aggregate_results(eval_magma_auto, "eval_results/magma_auto.csv")
results_magma_avg = aggregate_results(eval_magma_avg, "eval_results/magma_avg.csv")
results_magma_full = aggregate_results(eval_magma_full, "eval_results/magma_full.csv")

No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
No keypoints detected.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches were found.
Not enough

In [ ]:
eval_viridis_full = main_routine(get_cmap_full, "viridis", f"../datasets/stitches_viridis_full")
eval_viridis_avg = main_routine(get_cmap_avg, "viridis", f"../datasets/stitches_viridis_avg")
eval_viridis_auto = main_routine(get_cmap_auto, "viridis", f"../datasets/stitches_viridis_auto")
results_viridis_auto = aggregate_results(eval_viridis_auto, "eval_results/viridis_auto.csv")
results_viridis_avg = aggregate_results(eval_viridis_avg, "eval_results/viridis_avg.csv")
results_viridis_full = aggregate_results(eval_viridis_full, "eval_results/viridis_full.csv")

No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No 

In [ ]:
eval_cividis_full = main_routine(get_cmap_full, "cividis", f"../datasets/stitches_cividis_full")
eval_cividis_avg = main_routine(get_cmap_avg, "cividis", f"../datasets/stitches_cividis_avg")
eval_cividis_auto = main_routine(get_cmap_auto, "cividis", f"../datasets/stitches_cividis_auto")
results_cividis_auto = aggregate_results(eval_cividis_auto, "eval_results/cividis_auto.csv")
results_cividis_avg = aggregate_results(eval_cividis_avg, "eval_results/cividis_avg.csv")
results_cividis_full = aggregate_results(eval_cividis_full, "eval_results/cividis_full.csv")

No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
No keypoints detected.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
No keypoints detected.
No keypoints detected.
No keypoints 

In [ ]:
eval_hsv_full = main_routine(get_cmap_full, "hsv", f"../datasets/stitches_hsv_full")
eval_hsv_avg = main_routine(get_cmap_avg, "hsv", f"../datasets/stitches_hsv_avg")
eval_hsv_auto = main_routine(get_cmap_auto, "hsv", f"../datasets/stitches_hsv_auto")
results_hsv_auto = aggregate_results(eval_hsv_auto, "eval_results/hsv_auto.csv")
results_hsv_avg = aggregate_results(eval_hsv_avg, "eval_results/hsv_avg.csv")
results_hsv_full = aggregate_results(eval_hsv_full, "eval_results/hsv_full.csv")

Not enough good matches were found.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
N

In [ ]:
eval_twilight_full = main_routine(get_cmap_full, "twilight", f"../datasets/stitches_twilight_full")
eval_twilight_avg = main_routine(get_cmap_avg, "twilight", f"../datasets/stitches_twilight_avg")
eval_twilight_auto = main_routine(get_cmap_auto, "twilight", f"../datasets/stitches_twilight_auto")
results_twilight_auto = aggregate_results(eval_twilight_auto, "eval_results/twilight_auto.csv")
results_twilight_avg = aggregate_results(eval_twilight_avg, "eval_results/twilight_avg.csv")
results_twilight_full = aggregate_results(eval_twilight_full, "eval_results/twilight_full.csv")

Not enough good matches were found.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches wer

In [ ]:
eval_twilight_shifted_full = main_routine(get_cmap_full, "twilight_shifted", f"../datasets/stitches_twilight_shifted_full")
eval_twilight_shifted_avg = main_routine(get_cmap_avg, "twilight_shifted", f"../datasets/stitches_twilight_shifted_avg")
eval_twilight_shifted_auto = main_routine(get_cmap_auto, "twilight_shifted", f"../datasets/stitches_twilight_shifted_auto")
results_twilight_shifted_auto = aggregate_results(eval_twilight_shifted_auto, "eval_results/twilight_shifted_auto.csv")
results_twilight_shifted_avg = aggregate_results(eval_twilight_shifted_avg, "eval_results/twilight_shifted_avg.csv")
results_twilight_shifted_full = aggregate_results(eval_twilight_shifted_full, "eval_results/twilight_shifted_full.csv")

Not enough good matches were found.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found this iteration; skipped
No keypoints detected.
No keyp

In [ ]:
eval_CMRmap_full = main_routine(get_cmap_full, "CMRmap", f"../datasets/stitches_CMRmap_full")
eval_CMRmap_avg = main_routine(get_cmap_avg, "CMRmap", f"../datasets/stitches_CMRmap_avg")
eval_CMRmap_auto = main_routine(get_cmap_auto, "CMRmap", f"../datasets/stitches_CMRmap_auto")
results_CMRmap_auto = aggregate_results(eval_CMRmap_auto, "eval_results/CMRmap_auto.csv")
results_CMRmap_avg = aggregate_results(eval_CMRmap_avg, "eval_results/CMRmap_avg.csv")
results_CMRmap_full = aggregate_results(eval_CMRmap_full, "eval_results/CMRmap_full.csv")

Not enough good matches were found.
No keypoints detected.
No keypoints detected.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found this ite

In [ ]:
eval_Greys_full = main_routine(get_cmap_full, "Greys", f"../datasets/stitches_Greys_full")
eval_Greys_avg = main_routine(get_cmap_avg, "Greys", f"../datasets/stitches_Greys_avg")
eval_Greys_auto = main_routine(get_cmap_auto, "Greys", f"../datasets/stitches_Greys_auto")
results_Greys_auto = aggregate_results(eval_Greys_auto, "eval_results/Greys_auto.csv")
results_Greys_avg = aggregate_results(eval_Greys_avg, "eval_results/Greys_avg.csv")
results_Greys_full = aggregate_results(eval_Greys_full, "eval_results/Greys_full.csv")

No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found.
Not enough good matches were found.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not enough good matches were found.
Not enough good matches were found this iteration; skipped
Not enough good matches were found this iteration; skipped
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
No keypoints detected.
Not

In [12]:
# all_cmaps = ["hot", "inferno", "jet", "magma", "plasma", "viridis", "cividis", "hsv", "twilight_shifted", "twilight", "CMRmap"]

# for cmap in all_cmaps[8:]:
#     for dir in dirs:
#         _, rawdata = extract_paths(dir)
#         _ = color_map(dir, rawdata, get_cmap_auto, cmap, path_suf = "_focused")
#         _ = color_map(dir, rawdata, get_cmap_avg, cmap, path_suf = "_avg")
#         _ = color_map(dir, rawdata, get_cmap_full, cmap, path_suf = "_full")
